# Script notebook

A simple notebook to test script logic for downloading the data and plotting it. 

### Imports dependecies

In [5]:
import pathlib
import pandas as pd
import os
from googleapiclient.discovery import build
from datetime import datetime

### Load video IDs from a file

A `sources.csv` file holds a table of video ids for each contestant. 

In [6]:
current_path = pathlib.Path().absolute()
df = pd.read_csv(str(current_path) + '\\sources.csv', sep=',')
ids = df['key'].str.cat(sep=',')

### Fetch data from YouTube

Get current views statistics for each contestants using YouTube API

In [7]:
api_key = os.getenv('YOUTUBE_API_KEY')
youtube = build('youtube', 'v3', developerKey=api_key)
request = youtube.videos().list(id=ids, part='statistics')
response = request.execute()

### Append data file

Add current views data to a `data.csv` file, where all historical data are stored.

In [8]:
new_line = [datetime.now().strftime("%d/%m/%Y %H:%M:%S")]

for index, row in df.iterrows():
    new_line.append(next((item for item in response['items'] if item['id'] == df['key'].at[index]), None)['statistics']['viewCount'])

new_line_string = ','.join(map(str, new_line))

with open(str(current_path) + '\\data.csv', 'a') as data:
    data.write(f'\n{new_line_string}')


### Plot a chart

A chart that shows how a number a views of each contestant changed in time. The one who will get the most views at the final day of the contest (Eurovision 2021 - 22 May 2021) will be probably the winner.

In [9]:
df2 = pd.read_csv(str(current_path) + '\\data.csv', sep=',')

pd.options.plotting.backend = "plotly"
df2.plot.line()

ValueError: Plotly Express cannot process wide-form data with columns of different type.

### Delete last line in `data.csv` file

Its just for testing, delete last line in `data.csv` file.